Get Buffer Pool Consumers

In [2]:
SET NOCOUNT ON;
SET ANSI_WARNINGS ON;
SET QUOTED_IDENTIFIER ON;

DECLARE @sqlcmd NVARCHAR(max), @params NVARCHAR(600)
DECLARE @sqlmajorver int

SELECT @sqlmajorver = CONVERT(int, (@@microsoftversion / 0x1000000) & 0xff);

-- Note: in case of NUMA architecture, more than one entry per database is expected

SET @sqlcmd = 'SELECT ''Memory_checks'' AS [Category], ''Buffer_Pool_Consumers'' AS [Information], 
numa_node, COUNT_BIG(DISTINCT page_id)*8/1024 AS total_pages_MB, 
CASE database_id WHEN 32767 THEN ''ResourceDB'' ELSE DB_NAME(database_id) END AS database_name,
SUM(CONVERT(BIGINT,row_count))/COUNT_BIG(DISTINCT page_id) AS avg_row_count_per_page, 
SUM(CONVERT(BIGINT, free_space_in_bytes))/COUNT_BIG(DISTINCT page_id) AS avg_free_space_bytes_per_page
' + CASE WHEN @sqlmajorver >= 12 THEN ',is_in_bpool_extension' ELSE '' END + '
' + CASE WHEN @sqlmajorver = 10 THEN ',numa_node' ELSE '' END + '
' + CASE WHEN @sqlmajorver >= 11 THEN ',AVG(read_microsec) AS avg_read_microsec' ELSE '' END + '
FROM sys.dm_os_buffer_descriptors
--WHERE bd.page_type IN (''DATA_PAGE'', ''INDEX_PAGE'')
GROUP BY database_id' + CASE WHEN @sqlmajorver >= 10 THEN ', numa_node' ELSE '' END + CASE WHEN @sqlmajorver >= 12 THEN ', is_in_bpool_extension' ELSE '' END + '
ORDER BY total_pages_MB DESC;'
EXECUTE sp_executesql @sqlcmd;


Commands completed successfully.

Total execution time: 00:00:00.030

Category,Information,numa_node,total_pages_MB,database_name,avg_row_count_per_page,avg_free_space_bytes_per_page,is_in_bpool_extension,avg_read_microsec
Memory_checks,Buffer_Pool_Consumers,0,22,ResourceDB,20,2666,NULL,731
Memory_checks,Buffer_Pool_Consumers,0,4,SSISDB,36,3837,NULL,1573
Memory_checks,Buffer_Pool_Consumers,0,3,master,41,3713,NULL,668
Memory_checks,Buffer_Pool_Consumers,0,2,msdb,53,4024,NULL,637
Memory_checks,Buffer_Pool_Consumers,0,2,HappyScoopers_Demo,40,3159,NULL,890
Memory_checks,Buffer_Pool_Consumers,0,2,HappyScoopers_DW,36,2979,NULL,3436
Memory_checks,Buffer_Pool_Consumers,0,2,ns_lots_of_vlfs,47,3330,NULL,642
Memory_checks,Buffer_Pool_Consumers,0,2,my_bloated_db,48,3254,NULL,581
Memory_checks,Buffer_Pool_Consumers,0,2,car_crash,40,2913,NULL,1822
Memory_checks,Buffer_Pool_Consumers,0,1,tempdb,77,3920,NULL,1016
